In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
from datetime import datetime

# Configura o Chrome para rodar em modo headless
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

# Acessa o site da B3
url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"
driver.get(url)
time.sleep(5)  # Espera o JS carregar

# Inicializa lista de DataFrames
all_dfs = []

while True:
    # Captura a tabela da página atual
    html = driver.page_source
    dfs = pd.read_html(html)
    if dfs:
        all_dfs.append(dfs[0])

    # Tenta clicar no botão "Próxima"
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "next")]')
        # Verifica se está desabilitado
        if "disabled" in next_button.get_attribute("class"):
            break  # Acabou a paginação
        next_button.click()
        time.sleep(3)  # Espera o conteúdo atualizar
    except Exception as e:
        print("Erro ao tentar avançar página:", e)
        break

# Fecha o navegador
driver.quit()

# Concatena todos os DataFrames
df_final = pd.concat(all_dfs, ignore_index=True)

# Salva como Parquet
data_hoje = datetime.today().strftime("%Y-%m-%d")
df_final.to_parquet(f"b3_raw_{data_hoje}.parquet", index=False)
print(f"Salvo como: b3_raw_{data_hoje}.parquet")


In [ ]:
df = pd.read_parquet(r"C:\Users\Lukas\OneDrive\Documentos\Projetos FIAP\tech-challenge-2\b3_raw_2025-07-03.parquet")
df

In [11]:
import requests
import pandas as pd
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
import io

In [14]:
import requests
import pandas as pd
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq
import os

# 1. Faz o scraping dos dados da B3
def fetch_b3_data():
    url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"
    headers = {"User-Agent": "Mozilla/5.0"}

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Erro ao acessar a página: {response.status_code}")
    
    tables = pd.read_html(response.text)
    df = tables[0]  # supondo que a primeira tabela seja a desejada
    return df

# 2. Converte e salva localmente como Parquet
def save_to_parquet(df):
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")

    output_dir = f"b3_parquet_output/data_processamento={date_str}"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "b3_pregao.parquet")

    table = pa.Table.from_pandas(df)
    pq.write_table(table, output_path)

    print(f"Arquivo salvo em: {output_path}")
    return output_path

# 3. Executa tudo
df_b3 = fetch_b3_data()
print("Prévia dos dados:")
display(df_b3.head())

parquet_path = save_to_parquet(df_b3)


C:\Users\Lukas\AppData\Local\Temp\ipykernel_19328\1782075717.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [ ]:
import pandas as pd
df = pd.read_parquet(r'C:\Users\Lukas\OneDrive\Documentos\Projetos FIAP\tech-challenge-2\b3_raw_2025-07-03.parquet')
df['Qtde. Teórica'] = (
    df['Qtde. Teórica']
    .astype(str)                      # transforma em string
    .str.replace('.', '', regex=False)   # remove pontos de milhar
    .str.split(',', n=1).str[0]      # pega só a parte inteira antes da vírgula
    .astype(int)                     # converte para inteiro
)
df.to_parquet(r'C:\Users\Lukas\OneDrive\Documentos\Projetos FIAP\tech-challenge-2\b3_raw_2025-07-03.parquet')


In [17]:
pd.read_parquet(r'C:\Users\Lukas\OneDrive\Documentos\Projetos FIAP\tech-challenge-2\b3_parquet_output\data_processamento=2025-07-15\b3_portfolio_ibov.parquet')

,cod,asset,type,theoricalQty,part,cont
0,ABEV3,AMBEV S/A,ON,4.394.835.131,"2,880",2
1,ALOS3,ALLOS,ON ED NM,476.976.044,"0,477",1
2,ASAI3,ASSAI,ON ED NM,1.345.897.506,"0,573",3
3,AURE3,AUREN,ON ED NM,323.738.747,"0,135",4
4,AZUL4,AZUL,PN N2,886.824.173,"0,061",5
...,...,...,...,...,...,...
82,VBBR3,VIBRA,ON NM,1.113.939.036,"0,973",84
83,VIVA3,VIVARA S.A.,ON ED NM,123.160.591,"0,126",85
84,VIVT3,TELEF BRASIL,ON,764.884.256,"0,971",77
85,WEGE3,WEG,ON NM,1.482.105.837,"3,056",86
